In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec
from collections import defaultdict

In [3]:
df = pd.read_csv(r"C:\Users\stjia\Desktop\Coding\Computational Data Science_2024\Project\commodity-prices.csv")


In [4]:
df.head()

,Date,Commodity,Price index
0,1994-10,China import Iron Ore Fines 62% FE spot,11.450000
1,1994-10,Cotton,73.875000
2,1994-10,Groundnuts peanuts,870.000000
3,1994-11,Rapeseed oil,654.940000
4,1994-12,Sunflower oil,688.386364


In [5]:
df["Year"]=df["Date"].apply(lambda x: x.split("-")[0])

df

,Date,Commodity,Price index,Year
0,1994-10,China import Iron Ore Fines 62% FE spot,11.450000,1994
1,1994-10,Cotton,73.875000,1994
2,1994-10,Groundnuts peanuts,870.000000,1994
3,1994-11,Rapeseed oil,654.940000,1994
4,1994-12,Sunflower oil,688.386364,1994
...,...,...,...,...
26185,2016-02,Soft Sawnwood,310.554000,2016
26186,2016-02,Olive Oil,4545.695253,2016
26187,2016-02,Sunflower oil,1039.578571,2016
26188,2016-02,Tin,15610.142860,2016


In [6]:
columns=df.columns.to_list()

columns

['Date', 'Commodity', 'Price index', 'Year']

In [7]:
df=df[columns[-1:]+columns[:-1]]

df

,Year,Date,Commodity,Price index
0,1994,1994-10,China import Iron Ore Fines 62% FE spot,11.450000
1,1994,1994-10,Cotton,73.875000
2,1994,1994-10,Groundnuts peanuts,870.000000
3,1994,1994-11,Rapeseed oil,654.940000
4,1994,1994-12,Sunflower oil,688.386364
...,...,...,...,...
26185,2016,2016-02,Soft Sawnwood,310.554000
26186,2016,2016-02,Olive Oil,4545.695253
26187,2016,2016-02,Sunflower oil,1039.578571
26188,2016,2016-02,Tin,15610.142860


In [8]:
pd.to_datetime(df["Date"])



0       1994-10-01
1       1994-10-01
2       1994-10-01
3       1994-11-01
4       1994-12-01
           ...    
26185   2016-02-01
26186   2016-02-01
26187   2016-02-01
26188   2016-02-01
26189   2016-02-01
Name: Date, Length: 26190, dtype: datetime64[ns]

In [9]:
df.sort_values(by=['Date'], inplace=True)
print(df)

       Year     Date                      Commodity  Price index
8539   1980  1980-01                    Cocoa beans  3167.156982
3254   1980  1980-01             Metals Price Index    84.049004
3253   1980  1980-01  Industrial Inputs Price Index    81.889650
3252   1980  1980-01                         Nickel  6584.800781
3251   1980  1980-01                   Rapeseed oil   591.590000
...     ...      ...                            ...          ...
5651   2016  2016-02                      Wool fine  1017.063038
9749   2016  2016-02                   Soybean Meal   291.368437
4711   2016  2016-02    Sugar European import price    23.770766
4492   2016  2016-02                 Coffee Robusta    81.158000
26189  2016  2016-02  Food and Beverage Price Index   136.995434

[26190 rows x 4 columns]


In [10]:
unique_commodities_1980=df[df["Year"]=="1980"]["Commodity"].unique()
print(len(unique_commodities_1980))


54


In [11]:
filtered_df=df[df["Commodity"].isin(unique_commodities_1980)]
filtered_df.reset_index(drop=True, inplace=True)

filtered_df

,Year,Date,Commodity,Price index
0,1980,1980-01,Cocoa beans,3167.156982
1,1980,1980-01,Metals Price Index,84.049004
2,1980,1980-01,Industrial Inputs Price Index,81.889650
3,1980,1980-01,Nickel,6584.800781
4,1980,1980-01,Rapeseed oil,591.590000
...,...,...,...,...
23431,2016,2016-02,Oranges,691.367785
23432,2016,2016-02,Barley,120.116796
23433,2016,2016-02,Wool fine,1017.063038
23434,2016,2016-02,Soybean Meal,291.368437


In [12]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller


wheat = filtered_df[filtered_df['Commodity'] == "Wheat"]['Price index']
cocoa_beans = filtered_df[filtered_df['Commodity'] == "Cocoa beans"]['Price index']

result_wheat=adfuller(wheat)
result_cocoa_beans=adfuller(cocoa_beans)

print('ADF Statistic for Wheat: %f' % result_wheat[0])
print('p-value: %f' % result_wheat[1])
print('ADF Statistic for Cocoa Beans: %f' % result_cocoa_beans[0])
print('p-value: %f' % result_cocoa_beans[1])


ADF Statistic for Wheat: -2.512479
p-value: 0.112468
ADF Statistic for Cocoa Beans: -2.024964
p-value: 0.275736


In [13]:
# Differencing Wheat and Cocoa Beans series
wheat_1st_diff = wheat.diff()
cocoa_beans_1st_diff = cocoa_beans.diff()

# Dropping the first NaN value that results from differencing
wheat_1st_diff = wheat_1st_diff.dropna()
cocoa_beans_1st_diff = cocoa_beans_1st_diff.dropna()

result_wheat_diff = adfuller(wheat_1st_diff)
result_cocoabeans_diff = adfuller(cocoa_beans_1st_diff)

print('ADF Statistic for differenced Wheat:', result_wheat_diff[0])
print('p-value for differenced Wheat:', result_wheat_diff[1])

print('ADF Statistic for differenced Cocoa Beans:', result_cocoabeans_diff[0])
print('p-value for differenced Cocoa Beans:', result_cocoabeans_diff[1])


ADF Statistic for differenced Wheat: -8.200270594506291
p-value for differenced Wheat: 7.262403102538537e-13
ADF Statistic for differenced Cocoa Beans: -14.637016723770042
p-value for differenced Cocoa Beans: 3.674575581713426e-27


In [14]:
import statsmodels.api as sm

X = sm.add_constant(wheat_1st_diff) 
X.reset_index(drop=True, inplace=True)
cocoa_beans_1st_diff.reset_index(drop=True, inplace=True)

model = sm.OLS(cocoa_beans_1st_diff, X).fit()


residual = model.resid

residual_result = adfuller(residual)

print('ADF Statistic for residuals:', residual_result[0])
print('p-value for residuals:', residual_result[1])


ADF Statistic for residuals: -14.405168107336378
p-value for residuals: 8.381047237967284e-27


Looking for non-stationary commodities to prepare for Engle-Granger
* Also testing for i(1) by differentiating and ADF-test


In [15]:
list_of_commodities = unique_commodities_1980.tolist()
stationary_commodities = []

for commodity in list_of_commodities: 
    result=adfuller(filtered_df[filtered_df['Commodity'] == commodity]['Price index'])
    if result[1]<0.05:
        stationary_commodities.append(commodity)
print(stationary_commodities)

list_of_commodities = [commodity for commodity in list_of_commodities if commodity not in stationary_commodities]

['Coffee Other Mild Arabicas', 'Zinc', 'Sunflower oil', 'Wool fine', 'Cotton', 'Sugar Free Market', 'Shrimp', 'Sugar U.S. import price', 'Swine - pork', 'Tea', 'Aluminum']


In [16]:
print(len(list_of_commodities))

43


Running Engle-Granger on Wheat 

In [17]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Assuming 'filtered_df' is your DataFrame with 'Date' as index and commodities as columns

# Select two commodities to test for cointegration
wheat = filtered_df[filtered_df['Commodity'] == "Wheat"]['Price index']

cointegrated_list=[]


for commodity in list_of_commodities:
    wheat_1st_diff = wheat.diff()
    commodity_1st_diff=filtered_df[filtered_df['Commodity'] == commodity]['Price index'].diff()
    wheat_1st_diff = wheat_1st_diff.dropna()
    commodity_1st_diff = commodity_1st_diff.dropna()
    result_wheat_diff = adfuller(wheat_1st_diff)
    result_commoditity_diff = adfuller(commodity_1st_diff)

    if result_wheat_diff[1]<0.05 and result_commoditity_diff[1]<0.05:
        X = sm.add_constant(wheat_1st_diff)  # Adds a constant term to the independent variable
        X.reset_index(drop=True, inplace=True)
        commodity_1st_diff.reset_index(drop=True, inplace=True)
        model = sm.OLS(commodity_1st_diff, X).fit()  # OLS regression
        residuals = model.resid  # Get the residuals

        #  Test for Stationarity of Residuals
        adf_result = adfuller(residuals)
        if adf_result[1] < 0.05:
            cointegrated_list.append(commodity)

print(cointegrated_list)


['Cocoa beans', 'Metals Price Index', 'Industrial Inputs Price Index', 'Nickel', 'Rapeseed oil', 'Tin', 'Beef', 'Oil Dubai', 'Coffee Robusta', 'Groundnuts peanuts', 'Soybeans', 'Bananas', 'Lead', 'Soft Logs', 'Wool coarse', 'Coal', 'Barley', 'Uranium', 'Crude Oil - petroleum-simple average of three spot prices', 'Crude Oil petroleum', 'Hard Sawnwood', 'Olive Oil', 'China import Iron Ore Fines 62% FE spot', 'Crude Oil - petroleum - Dated Brent light blend', 'Rice', 'Hard Logs', 'Soft Sawnwood', 'Palm oil', 'Copper', 'Agricultural Raw Materials Index', 'Soybean Meal', 'Oranges', 'Soybean Oil', 'Fish salmon', 'Hides', 'Poultry chicken', 'Rubber', 'Maize corn', 'Lamb', 'Crude Oil petroleum - West Texas Intermediate 40 API', 'Fishmeal', 'Beverage Price Index']


In [18]:
print(len(cointegrated_list))

42


In [19]:
import yfinance as yf
# Define the commodity ticker symbol on Yahoo Finance. For example, 'GC=F' for gold futures.
ticker_symbol = 'GC=F'

# Fetch historical data
total_commodity_data = yf.download(ticker_symbol, start='2018-01-01', end='2022-12-31', interval='1d')

total_commodity_data = total_commodity_data=total_commodity_data.rename(columns=lambda x: 'gold_futures_' + x if x != 'Date' else x)


[*********************100%%**********************]  1 of 1 completed


In [20]:
tickers=["ZT=F",
"ZS=F",
"ZR=F",
"ZO=F",
"ZN=F",
"ZM=F",
"ZL=F",
"ZF=F",
"ZC=F",
"ZB=F",
"YM=F",
"SIL=F",
"SI=F",
"SB=F",
"RTY=F",
"RB=F",
"PL=F",
"PA=F",
"OJ=F",
"NQ=F",
"NG=F",
"MGC=F",
"LE=F",
"LBS=F",
"KE=F",
"KC=F",
"HO=F",
"HG=F",
"HE=F",
"GF=F",
"ES=F",
"CT=F",
"CL=F",
"CC=F",
"BZ=F"
]

In [21]:
for ticker_symbol in tickers:
    commodity_data = yf.download(ticker_symbol, start='2018-01-01', end='2022-12-31', interval='1d')
    commodity_data = commodity_data.rename(columns=lambda x: ticker_symbol + '_' + x if x != 'Date' else x)
    total_commodity_data = total_commodity_data.merge(commodity_data, on='Date', how='outer')

total_commodity_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,gold_futures_Open,gold_futures_High,gold_futures_Low,gold_futures_Close,gold_futures_Adj Close,gold_futures_Volume,ZT=F_Open,ZT=F_High,ZT=F_Low,ZT=F_Close,...,CC=F_Low,CC=F_Close,CC=F_Adj Close,CC=F_Volume,BZ=F_Open,BZ=F_High,BZ=F_Low,BZ=F_Close,BZ=F_Adj Close,BZ=F_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,1302.300049,1317.599976,1302.300049,1313.699951,1313.699951,68.0,107.046875,107.054688,106.968750,106.984375,...,1862.0,1936.0,1936.0,26852,66.419998,67.279999,66.230003,66.570000,66.570000,24294.0
2018-01-03,1320.000000,1320.099976,1312.099976,1316.199951,1316.199951,42.0,106.984375,107.007812,106.953125,106.968750,...,1888.0,1907.0,1907.0,21847,66.489998,68.010002,66.400002,67.839996,67.839996,25878.0
2018-01-04,1319.400024,1322.000000,1319.400024,1319.400024,1319.400024,2.0,106.953125,106.960938,106.875000,106.914062,...,1836.0,1905.0,1905.0,30322,68.000000,68.260002,67.529999,68.070000,68.070000,22840.0
2018-01-05,1320.300049,1320.300049,1320.300049,1320.300049,1320.300049,1.0,106.921875,106.968750,106.890625,106.906250,...,1866.0,1895.0,1895.0,14402,68.059998,68.089996,67.269997,67.620003,67.620003,18290.0
2018-01-08,1319.900024,1319.900024,1314.599976,1318.599976,1318.599976,41.0,106.898438,106.929688,106.890625,106.914062,...,1870.0,1914.0,1914.0,21988,67.820000,67.980003,67.449997,67.779999,67.779999,17704.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,1794.300049,1802.800049,1794.099976,1795.900024,1795.900024,49.0,102.695312,102.695312,102.679688,102.695312,...,2609.0,2628.0,2628.0,26051,81.730003,84.370003,81.339996,83.919998,83.919998,8621.0
2022-12-27,1803.400024,1826.300049,1803.400024,1814.800049,1814.800049,69.0,102.609375,102.609375,102.484375,102.507812,...,2616.0,2633.0,2633.0,16555,84.459999,85.669998,83.660004,84.330002,84.330002,7512.0
2022-12-28,1803.199951,1807.900024,1803.099976,1807.900024,1807.900024,434.0,102.500000,102.656250,102.500000,102.535156,...,2604.0,2621.0,2621.0,20880,84.599998,84.639999,81.949997,83.260002,83.260002,5384.0


In [22]:
total_commodity_data.to_csv('one_day_historical_data.csv')

In [23]:
import pandas as pd

# Temporarily set option to display more rows
pd.set_option('display.max_rows', 500)
null_counts = total_commodity_data.isnull().sum()
print(null_counts)

# Reset option back to default
pd.reset_option('display.max_rows')




gold_futures_Open           1
gold_futures_High           1
gold_futures_Low            1
gold_futures_Close          1
gold_futures_Adj Close      1
gold_futures_Volume         1
ZT=F_Open                   0
ZT=F_High                   0
ZT=F_Low                    0
ZT=F_Close                  0
ZT=F_Adj Close              0
ZT=F_Volume                 0
ZS=F_Open                   0
ZS=F_High                   0
ZS=F_Low                    0
ZS=F_Close                  0
ZS=F_Adj Close              0
ZS=F_Volume                 0
ZR=F_Open                  12
ZR=F_High                  12
ZR=F_Low                   12
ZR=F_Close                 12
ZR=F_Adj Close             12
ZR=F_Volume                12
ZO=F_Open                   0
ZO=F_High                   0
ZO=F_Low                    0
ZO=F_Close                  0
ZO=F_Adj Close              0
ZO=F_Volume                 0
ZN=F_Open                   0
ZN=F_High                   0
ZN=F_Low                    0
ZN=F_Close

In [24]:
columns_to_drop = [col for col in total_commodity_data.columns if 'OJ=F' in col]
total_commodity_data = total_commodity_data.drop(columns_to_drop, axis=1)

In [25]:
total_commodity_data_cleaned = total_commodity_data.dropna()


In [26]:
total_commodity_data_cleaned.to_csv('one_day_historical_data.csv')

In [27]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [40.6331, 41.878],
	"longitude": [89.3985, 93.0977],
	"start_date": "2018-01-01",
	"end_date": "2022-12-31",
	"daily": ["temperature_2m_mean", "sunshine_duration", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"timezone": "Asia/Singapore"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(1).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(2).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(3).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(4).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(5).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(6).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 40.6678352355957°N 89.36260223388672°E
Elevation 804.0 m asl
Timezone b'Asia/Singapore' b'+08'
Timezone difference to GMT+0 28800 s
                          date  temperature_2m_mean  sunshine_duration  \
0    2017-12-31 16:00:00+00:00            -4.643249       23525.304688   
1    2018-01-01 16:00:00+00:00            -4.828666       28800.000000   
2    2018-01-02 16:00:00+00:00            -7.022415       28915.564453   
3    2018-01-03 16:00:00+00:00            -9.522416       28800.000000   
4    2018-01-04 16:00:00+00:00            -9.303668       29004.486328   
...                        ...                  ...                ...   
1821 2022-12-26 16:00:00+00:00            -9.339084       28800.000000   
1822 2022-12-27 16:00:00+00:00            -9.816167       28800.000000   
1823 2022-12-28 16:00:00+00:00           -10.503666       28800.000000   
1824 2022-12-29 16:00:00+00:00           -10.459917       28800.000000   
1825 2022-12-30 16:00:00+00:00            

In [28]:
daily_dataframe['date']=pd.to_datetime(daily_dataframe['date'])
daily_dataframe['date']=daily_dataframe['date'].dt.strftime('%m-%d-%Y')

In [29]:
daily_dataframe['Date']=daily_dataframe['date']

In [30]:
daily_dataframe.to_csv('daily_weather_data.csv')

In [31]:
total_commodity_data_cleaned

,gold_futures_Open,gold_futures_High,gold_futures_Low,gold_futures_Close,gold_futures_Adj Close,gold_futures_Volume,ZT=F_Open,ZT=F_High,ZT=F_Low,ZT=F_Close,...,CC=F_Low,CC=F_Close,CC=F_Adj Close,CC=F_Volume,BZ=F_Open,BZ=F_High,BZ=F_Low,BZ=F_Close,BZ=F_Adj Close,BZ=F_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,1302.300049,1317.599976,1302.300049,1313.699951,1313.699951,68.0,107.046875,107.054688,106.968750,106.984375,...,1862.0,1936.0,1936.0,26852,66.419998,67.279999,66.230003,66.570000,66.570000,24294.0
2018-01-03,1320.000000,1320.099976,1312.099976,1316.199951,1316.199951,42.0,106.984375,107.007812,106.953125,106.968750,...,1888.0,1907.0,1907.0,21847,66.489998,68.010002,66.400002,67.839996,67.839996,25878.0
2018-01-04,1319.400024,1322.000000,1319.400024,1319.400024,1319.400024,2.0,106.953125,106.960938,106.875000,106.914062,...,1836.0,1905.0,1905.0,30322,68.000000,68.260002,67.529999,68.070000,68.070000,22840.0
2018-01-05,1320.300049,1320.300049,1320.300049,1320.300049,1320.300049,1.0,106.921875,106.968750,106.890625,106.906250,...,1866.0,1895.0,1895.0,14402,68.059998,68.089996,67.269997,67.620003,67.620003,18290.0
2018-01-08,1319.900024,1319.900024,1314.599976,1318.599976,1318.599976,41.0,106.898438,106.929688,106.890625,106.914062,...,1870.0,1914.0,1914.0,21988,67.820000,67.980003,67.449997,67.779999,67.779999,17704.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,1794.300049,1802.800049,1794.099976,1795.900024,1795.900024,49.0,102.695312,102.695312,102.679688,102.695312,...,2609.0,2628.0,2628.0,26051,81.730003,84.370003,81.339996,83.919998,83.919998,8621.0
2022-12-27,1803.400024,1826.300049,1803.400024,1814.800049,1814.800049,69.0,102.609375,102.609375,102.484375,102.507812,...,2616.0,2633.0,2633.0,16555,84.459999,85.669998,83.660004,84.330002,84.330002,7512.0
2022-12-28,1803.199951,1807.900024,1803.099976,1807.900024,1807.900024,434.0,102.500000,102.656250,102.500000,102.535156,...,2604.0,2621.0,2621.0,20880,84.599998,84.639999,81.949997,83.260002,83.260002,5384.0


In [32]:
daily_dataframe

,date,temperature_2m_mean,sunshine_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,Date
0,12-31-2017,-4.643249,23525.304688,28.054604,43.560001,66.283279,7.06,1.316033,12-31-2017
1,01-01-2018,-4.828666,28800.000000,27.708309,43.560001,57.318825,8.46,1.784355,01-01-2018
2,01-02-2018,-7.022415,28915.564453,23.535301,37.799999,68.721916,8.75,1.265068,01-02-2018
3,01-03-2018,-9.522416,28800.000000,17.174677,25.559999,67.445747,8.49,0.836272,01-03-2018
4,01-04-2018,-9.303668,29004.486328,20.696087,29.519999,52.933533,8.88,1.016240,01-04-2018
...,...,...,...,...,...,...,...,...,...
1821,12-26-2022,-9.339084,28800.000000,9.726665,16.559999,38.198669,8.16,0.616462,12-26-2022
1822,12-27-2022,-9.816167,28800.000000,10.587917,15.480000,45.673985,8.21,0.592717,12-27-2022
1823,12-28-2022,-10.503666,28800.000000,11.183201,18.359999,280.918915,8.25,0.582818,12-28-2022
1824,12-29-2022,-10.459917,28800.000000,18.837322,30.599998,297.823975,8.22,0.616115,12-29-2022


In [33]:
daily_dataframe

,date,temperature_2m_mean,sunshine_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,Date
0,12-31-2017,-4.643249,23525.304688,28.054604,43.560001,66.283279,7.06,1.316033,12-31-2017
1,01-01-2018,-4.828666,28800.000000,27.708309,43.560001,57.318825,8.46,1.784355,01-01-2018
2,01-02-2018,-7.022415,28915.564453,23.535301,37.799999,68.721916,8.75,1.265068,01-02-2018
3,01-03-2018,-9.522416,28800.000000,17.174677,25.559999,67.445747,8.49,0.836272,01-03-2018
4,01-04-2018,-9.303668,29004.486328,20.696087,29.519999,52.933533,8.88,1.016240,01-04-2018
...,...,...,...,...,...,...,...,...,...
1821,12-26-2022,-9.339084,28800.000000,9.726665,16.559999,38.198669,8.16,0.616462,12-26-2022
1822,12-27-2022,-9.816167,28800.000000,10.587917,15.480000,45.673985,8.21,0.592717,12-27-2022
1823,12-28-2022,-10.503666,28800.000000,11.183201,18.359999,280.918915,8.25,0.582818,12-28-2022
1824,12-29-2022,-10.459917,28800.000000,18.837322,30.599998,297.823975,8.22,0.616115,12-29-2022


In [34]:
daily_dataframe.to_csv('daily_weather_data.csv')

In [35]:
import pandas as pd

f = pd.read_csv(r"C:\Users\stjia\Desktop\Coding\Computational Data Science_2024\Project\combined_commodity_weather.csv")d

In [36]:
import pandas as pd

f = pd.read_csv(r"C:\Users\stjia\Desktop\Coding\Computational Data Science_2024\Project\combined_commodity_weather.csv")d

df['ZC=F_Open_Diff'] = df['ZC=F_Open'].diff()
df['ZC=F_Open_Diff'].fillna(0, inplace=True)


print(df["ZC=F_Open_Diff"])

df.drop('ZC=F_Open', axis=1, inplace=True)


C:\Users\stjia\AppData\Local\Temp\ipykernel_28344\2622092436.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ZC=F_Open_Diff'].fillna(0, inplace=True)


In [38]:
df["ZC=F_Open_Diff"].median()

0.25

In [39]:
df["ZC=F_Open_Diff"].quantile(0.75)

4.0

In [40]:
df['ZC=F_Open_Diff_absolute'] = df['ZC=F_Open_Diff'].apply(lambda x: 1 if x > 0 else 0)


df_cleaned=df.copy(deep=True)
df_cleaned['ZC=F_Open_Diff_absolute'] = df_cleaned['ZC=F_Open_Diff_absolute'].shift(-1)
df_cleaned.dropna(inplace=True)
df_cleaned.drop('ZC=F_Open_Diff', axis=1, inplace=True)
df_cleaned.drop('Date', axis=1, inplace=True)


In [32]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

# Assuming df_cleaned is already defined and preprocessed

scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_cleaned.drop('ZC=F_Open_Diff_absolute', axis=1))

X = np.reshape(features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1]))
y = df_cleaned['ZC=F_Open_Diff_absolute'].values

all_results = []  # This will store results for all model runs

for model_run in range(10):  # 10 model runs
    results = []  # This will store the results for the folds in the current model run
    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.2, recurrent_dropout=0.2),
            LSTM(50, dropout=0.2, recurrent_dropout=0.2),
            Dense(1, activation='sigmoid', kernel_regularizer=L1L2(l1=0.01, l2=0.01))
        ])

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=100, batch_size=4 * model_run + 4, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

        scores = model.evaluate(X_test, y_test, verbose=0)
        results.append(scores)

    all_results.append(results)  # Store the results of the current model run

# Print the results in a tabular format
print("Model No. | Fold | Loss     | Accuracy")
print("--------------------------------------")
for model_no, model_results in enumerate(all_results):
    for fold_no, score in enumerate(model_results):
        print(f"{model_no+1}        | {fold_no+1}    | {score[0]:.4f} | {score[1]:.4f}")

Epoch 1/100


c:\Users\stjia\anaconda3\envs\wheat_ml\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.4337 - loss: 0.8023 - val_accuracy: 0.4627 - val_loss: 0.7786
Epoch 2/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5021 - loss: 0.7699 - val_accuracy: 0.5323 - val_loss: 0.7538
Epoch 3/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5133 - loss: 0.7464 - val_accuracy: 0.4627 - val_loss: 0.7357
Epoch 4/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5495 - loss: 0.7289 - val_accuracy: 0.4776 - val_loss: 0.7211
Epoch 5/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6101 - loss: 0.7143 - val_accuracy: 0.5075 - val_loss: 0.7117
Epoch 6/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5634 - loss: 0.7072 - val_accuracy: 0.4776 - val_loss: 0.7051
Epoch 7/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6415 - loss: 0.6975 - val_accuracy: 0.4726 - val_loss: 0.7005
Epoch 8/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5208 - loss: 0.6970 - val_accuracy: 0.4378 - val_loss: 0.

In [60]:
tscv = TimeSeriesSplit(n_splits=5)

# Iterate over each split
for train_index, test_index in tscv.split(X):
    # Accessing the first and last element of each index array
    print("Train indices range: {} - {}".format(train_index[0], train_index[-1]))
    print("Test indices range: {} - {}".format(test_index[0], test_index[-1]))
    print("-" * 40)  # Just for better readability between splits

Train indices range: 0 - 204
Test indices range: 205 - 407
----------------------------------------
Train indices range: 0 - 407
Test indices range: 408 - 610
----------------------------------------
Train indices range: 0 - 610
Test indices range: 611 - 813
----------------------------------------
Train indices range: 0 - 813
Test indices range: 814 - 1016
----------------------------------------
Train indices range: 0 - 1016
Test indices range: 1017 - 1219
----------------------------------------


In [33]:
average_accuracy = []

for model_no, model_results in enumerate(all_results):
    total=0
    for model in model_results:
        total+=model[1]
    average_accuracy.append(total/5)

print(average_accuracy)

[0.509452736377716, 0.49850745797157286, 0.4845771133899689, 0.5024875700473785, 0.4925373136997223, 0.49950246810913085, 0.48955224752426146, 0.46666666865348816, 0.48656717538833616, 0.4965174078941345]


In [34]:
average_accuracy_2 = []
split_1=0
split_2=0
split_3=0
split_4=0
split_5=0

for model_no, model_results in enumerate(all_results):
    total=0
    for split, model in enumerate(model_results):
        if split==0:
            split_1+=model[1]
        elif split==1:
            split_2+=model[1]
        elif split==2:
            split_3+=model[1]
        elif split==3:
            split_4+=model[1]
        elif split==4:
            split_5+=model[1]

average_accuracy_2.append(split_1/10)
average_accuracy_2.append(split_2/10)
average_accuracy_2.append(split_3/10)
average_accuracy_2.append(split_4/10)
average_accuracy_2.append(split_5/10)

print(average_accuracy_2)

[0.48109452724456786, 0.5184079587459565, 0.3920397937297821, 0.541293528676033, 0.5303482711315155]


In [49]:
results = pd.DataFrame.from_records(all_results)
results.to_csv('results.csv', index=False)

In [42]:
model.evaluate(X_test, y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5419 - loss: 0.7014 


[0.700268566608429, 0.5320196747779846]

In [43]:
import numpy as np
from statsmodels.stats.diagnostic import acorr_ljungbox

# Assuming 'model' is your trained LSTM model and 'X_test' is your test dataset

# Step 1: Generate predictions
predictions = model.predict(X_test).flatten()  # Flatten if your predictions are in a 2D array

# Step 2: Calculate residuals
residuals = y_test - predictions

# Step 3: Conduct the Ljung-Box test on the residuals
# Test for autocorrelation up to lag 10 (you can adjust this number based on your dataset size)
ljung_box_result = acorr_ljungbox(residuals, lags=[1], return_df=True)

ljung_box_result


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step


,lb_stat,lb_pvalue
1,0.425236,0.514336


In [46]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Convert probabilities to binary predictions
binary_predictions = (predictions > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, binary_predictions)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, binary_predictions, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {fscore:.4f}")


Accuracy: 0.5320
Precision: 0.5320
Recall: 1.0000
F1-Score: 0.6945


In [65]:
print(X.shape)

(1220, 1, 216)


In [26]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

# Assuming df_cleaned is already defined and preprocessed

scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_cleaned.drop('ZC=F_Open_Diff_absolute', axis=1))

X = np.reshape(features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1]))
X_modified = np.concatenate((X[:567], X[934:]), axis=0)
X=X_modified
y = df_cleaned['ZC=F_Open_Diff_absolute'].values

all_results = []  # This will store results for all model runs

for model_run in range(10):  # 10 model runs
    results = []  # This will store the results for the folds in the current model run
    tscv = TimeSeriesSplit(n_splits=4)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.2, recurrent_dropout=0.2),
            LSTM(50, dropout=0.2, recurrent_dropout=0.2),
            Dense(1, activation='sigmoid', kernel_regularizer=L1L2(l1=0.01, l2=0.01))
        ])

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=100, batch_size=4 * model_run + 4, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

        scores = model.evaluate(X_test, y_test, verbose=0)
        results.append(scores)

    all_results.append(results)  # Store the results of the current model run

# Print the results in a tabular format
print("Model No. | Fold | Loss     | Accuracy")
print("--------------------------------------")
for model_no, model_results in enumerate(all_results):
    for fold_no, score in enumerate(model_results):
        print(f"{model_no+1}        | {fold_no+1}    | {score[0]:.4f} | {score[1]:.4f}")

Epoch 1/100


c:\Users\stjia\anaconda3\envs\wheat_ml\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5356 - loss: 0.7899 - val_accuracy: 0.4821 - val_loss: 0.7725
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5702 - loss: 0.7600 - val_accuracy: 0.4881 - val_loss: 0.7544
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5551 - loss: 0.7423 - val_accuracy: 0.4940 - val_loss: 0.7388
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5673 - loss: 0.7241 - val_accuracy: 0.5060 - val_loss: 0.7299
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5400 - loss: 0.7124 - val_accuracy: 0.4821 - val_loss: 0.7254
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6214 - loss: 0.6984 - val_accuracy: 0.4940 - val_loss: 0.7167
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5871 - loss: 0.6964 - val_accuracy: 0.5000 - val_loss: 0.7165
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5811 - loss: 0.6991 - val_accuracy: 0.4762 - val_loss: 0.

In [27]:
print(all_results)

[[[0.7197189927101135, 0.5178571343421936], [0.6918685436248779, 0.5952380895614624], [0.6956579685211182, 0.494047611951828], [0.6860416531562805, 0.5952380895614624]], [[0.739743173122406, 0.4761904776096344], [0.6936459541320801, 0.5773809552192688], [0.6959658265113831, 0.494047611951828], [0.6849458813667297, 0.5952380895614624]], [[0.7582581639289856, 0.4761904776096344], [0.6945702433586121, 0.494047611951828], [0.6972744464874268, 0.494047611951828], [0.6858966946601868, 0.5952380895614624]], [[0.7532405257225037, 0.494047611951828], [0.6995143890380859, 0.5], [0.6965295672416687, 0.494047611951828], [0.6874958872795105, 0.5952380895614624]], [[0.7616196274757385, 0.494047611951828], [0.7190156579017639, 0.3988095223903656], [0.7013322710990906, 0.494047611951828], [0.6880239844322205, 0.5952380895614624]], [[0.7696622014045715, 0.4821428656578064], [0.7174022197723389, 0.4464285671710968], [0.7011707425117493, 0.5535714030265808], [0.6891800761222839, 0.5952380895614624]], [[0

In [31]:
average_accuracy = []

for model_no, model_results in enumerate(all_results):
    total=0
    for model in model_results:
        total+=model[1]
    average_accuracy.append(total/4)

print(average_accuracy)

[0.5505952313542366, 0.5357142835855484, 0.5148809477686882, 0.5208333283662796, 0.495535708963871, 0.5193452313542366, 0.497023805975914, 0.5133928582072258, 0.495535708963871, 0.5178571417927742]


In [71]:
average_accuracy_2 = []
split_1=0
split_2=0
split_3=0
split_4=0


for model_no, model_results in enumerate(all_results):
    total=0
    for split, model in enumerate(model_results):
        if split==0:
            split_1+=model[1]
        elif split==1:
            split_2+=model[1]
        elif split==2:
            split_3+=model[1]
        elif split==3:
            split_4+=model[1]

average_accuracy_2.append(split_1/10)
average_accuracy_2.append(split_2/10)
average_accuracy_2.append(split_3/10)
average_accuracy_2.append(split_4/10)


print(average_accuracy_2)

[0.4941176503896713, 0.49882354140281676, 0.5041176676750183, 0.6000000238418579, 0.0]


In [12]:
import pandas as pd

df = pd.read_csv(r"C:\Users\stjia\Desktop\Coding\Computational Data Science_2024\Project\combined_commodity_weather.csv")

df['ZC=F_Open_Diff'] = df['ZC=F_Open'].diff()
df['ZC=F_Open_Diff'].fillna(0, inplace=True)



import pandas as pd
import numpy as np
import ta

# Assuming df is your DataFrame and 'ZC=F_Open' is the column with opening prices

# 1. Calculate EMA
ema_window = 14  # You can adjust the window size as needed
df['EMA'] = df["ZC=F_Open"].ewm(span=ema_window, adjust=False).mean()

# 2. Calculate RSI
rsi_window = 14  # You can adjust the window size as needed
df['RSI'] = ta.momentum.rsi(df["ZC=F_Open"], window=rsi_window)

# 3. Calculate Williams %R
williams_window = 14  # You can adjust the window size as needed
df['Williams_%R'] = ta.momentum.williams_r(high=df["ZC=F_Open"],  # Assuming high prices are in this column
                                           low=df["ZC=F_Open"],  # Assuming low prices are in this column
                                           close=df["ZC=F_Open"],
                                           lbp=williams_window)

df.drop('ZC=F_Open', axis=1, inplace=True)

df['ZC=F_Open_Diff_absolute'] = df['ZC=F_Open_Diff'].apply(lambda x: 1 if x > 0 else 0)


df_cleaned=df.copy(deep=True)
df_cleaned['ZC=F_Open_Diff_absolute'] = df_cleaned['ZC=F_Open_Diff_absolute'].shift(-1)
df_cleaned.dropna(inplace=True)
df_cleaned.drop('ZC=F_Open_Diff', axis=1, inplace=True)
df_cleaned.drop('Date', axis=1, inplace=True)



C:\Users\stjia\AppData\Local\Temp\ipykernel_17520\1898054967.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ZC=F_Open_Diff'].fillna(0, inplace=True)


In [13]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

# Assuming df_cleaned is already defined and preprocessed

scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_cleaned.drop('ZC=F_Open_Diff_absolute', axis=1))

X = np.reshape(features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1]))
y = df_cleaned['ZC=F_Open_Diff_absolute'].values

all_results = []  # This will store results for all model runs

for model_run in range(10):  # 10 model runs
    results = []  # This will store the results for the folds in the current model run
    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.2, recurrent_dropout=0.2),
            LSTM(50, dropout=0.2, recurrent_dropout=0.2),
            Dense(1, activation='sigmoid', kernel_regularizer=L1L2(l1=0.01, l2=0.01))
        ])

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=100, batch_size=4 * model_run + 4, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

        scores = model.evaluate(X_test, y_test, verbose=0)
        results.append(scores)

    all_results.append(results)  # Store the results of the current model run

# Print the results in a tabular format
print("Model No. | Fold | Loss     | Accuracy")
print("--------------------------------------")
for model_no, model_results in enumerate(all_results):
    for fold_no, score in enumerate(model_results):
        print(f"{model_no+1}        | {fold_no+1}    | {score[0]:.4f} | {score[1]:.4f}")

Epoch 1/100


c:\Users\stjia\anaconda3\envs\wheat_ml\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.4030 - loss: 0.7921 - val_accuracy: 0.4975 - val_loss: 0.7675
Epoch 2/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.7600 - val_accuracy: 0.4876 - val_loss: 0.7464
Epoch 3/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5348 - loss: 0.7388 - val_accuracy: 0.4677 - val_loss: 0.7326
Epoch 4/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5375 - loss: 0.7235 - val_accuracy: 0.4478 - val_loss: 0.7209
Epoch 5/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5833 - loss: 0.7122 - val_accuracy: 0.5373 - val_loss: 0.7113
Epoch 6/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5999 - loss: 0.7031 - val_accuracy: 0.4527 - val_loss: 0.7068
Epoch 7/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5625 - loss: 0.7003 - val_accuracy: 0.4677 - val_loss: 0.7027
Epoch 8/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6074 - loss: 0.6960 - val_accuracy: 0.4428 - val_loss: 0.

In [14]:
average_accuracy = []

for model_no, model_results in enumerate(all_results):
    total=0
    for model in model_results:
        total+=model[1]
    average_accuracy.append(total/5)

print(average_accuracy)

[0.5064676582813263, 0.5114427804946899, 0.5154228925704956, 0.4935323417186737, 0.48656716346740725, 0.49054725766181945, 0.5024875640869141, 0.48756218552589414, 0.4895522356033325, 0.4885572135448456]


In [15]:
average_accuracy_2 = []
split_1=0
split_2=0
split_3=0
split_4=0
split_5=0

for model_no, model_results in enumerate(all_results):
    total=0
    for split, model in enumerate(model_results):
        if split==0:
            split_1+=model[1]
        elif split==1:
            split_2+=model[1]
        elif split==2:
            split_3+=model[1]
        elif split==3:
            split_4+=model[1]
        elif split==4:
            split_5+=model[1]

average_accuracy_2.append(split_1/10)
average_accuracy_2.append(split_2/10)
average_accuracy_2.append(split_3/10)
average_accuracy_2.append(split_4/10)
average_accuracy_2.append(split_5/10)

print(average_accuracy_2)

[0.4776119410991669, 0.5358208984136581, 0.40199004411697387, 0.5422885477542877, 0.5283582150936127]


In [17]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

# Assuming df_cleaned is already defined and preprocessed

scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_cleaned.drop('ZC=F_Open_Diff_absolute', axis=1))

X = np.reshape(features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1]))
y = df_cleaned['ZC=F_Open_Diff_absolute'].values

all_results = []  # This will store results for all model runs


results = []  # This will store the results for the folds in the current model run
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.2, recurrent_dropout=0.2),
        LSTM(50, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid', kernel_regularizer=L1L2(l1=0.01, l2=0.01))
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

    scores = model.evaluate(X_test, y_test, verbose=0)
    results.append(scores)




Epoch 1/100


c:\Users\stjia\anaconda3\envs\wheat_ml\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5294 - loss: 0.8032 - val_accuracy: 0.5025 - val_loss: 0.7874
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5631 - loss: 0.7836 - val_accuracy: 0.4975 - val_loss: 0.7748
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5473 - loss: 0.7687 - val_accuracy: 0.4726 - val_loss: 0.7648
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6073 - loss: 0.7554 - val_accuracy: 0.5323 - val_loss: 0.7497
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5312 - loss: 0.7428 - val_accuracy: 0.5025 - val_loss: 0.7424
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5908 - loss: 0.7332 - val_accuracy: 0.5274 - val_loss: 0.7336
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6525 - loss: 0.7195 - val_accuracy: 0.4876 - val_loss: 0.7287
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6093 - loss: 0.7115 - val_accuracy: 0.4975 - val_loss: 0.

In [18]:
print(results)

[[0.7154836654663086, 0.5174129605293274], [0.6891066431999207, 0.5422885417938232], [0.707558274269104, 0.39303481578826904], [0.6963351368904114, 0.5621890425682068], [0.6953700184822083, 0.5323383212089539]]


In [21]:
# Print the results in a tabular format
print("Fold | Loss | Accuracy")
print("--------------------------------------")
for fold_no,score in enumerate(results):
    print(f"  {fold_no} | {score[0]:.4f} | {score[1]:.4f}")

Fold | Loss | Accuracy
--------------------------------------
  0 | 0.7155 | 0.5174
  1 | 0.6891 | 0.5423
  2 | 0.7076 | 0.3930
  3 | 0.6963 | 0.5622
  4 | 0.6954 | 0.5323


In [23]:
import numpy as np
from statsmodels.stats.diagnostic import acorr_ljungbox

# Assuming 'model' is your trained LSTM model and 'X_test' is your test dataset

# Step 1: Generate predictions
predictions = model.predict(X_test).flatten()  # Flatten if your predictions are in a 2D array

# Step 2: Calculate residuals
residuals = y_test - predictions

# Step 3: Conduct the Ljung-Box test on the residuals
# Test for autocorrelation up to lag 10 (you can adjust this number based on your dataset size)
ljung_box_result = acorr_ljungbox(residuals, lags=[1], return_df=True)

ljung_box_result


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,lb_stat,lb_pvalue
1,0.59465,0.440627


In [35]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

# Assuming df_cleaned is already defined and preprocessed

scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_cleaned.drop('ZC=F_Open_Diff_absolute', axis=1))

X = np.reshape(features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1]))
X_modified = np.concatenate((X[:567], X[934:]), axis=0)
X=X_modified
y = df_cleaned['ZC=F_Open_Diff_absolute'].values

all_results = []  # This will store results for all model runs


results = []  # This will store the results for the folds in the current model run
tscv = TimeSeriesSplit(n_splits=4)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.2, recurrent_dropout=0.2),
        LSTM(50, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid', kernel_regularizer=L1L2(l1=0.01, l2=0.01))
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=100, batch_size=4, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

    scores = model.evaluate(X_test, y_test, verbose=0)
    results.append(scores)

# Print the results in a tabular format
print("Fold | Loss | Accuracy")
print("--------------------------------------")
for fold_no,score in enumerate(results):
    print(f"  {fold_no} | {score[0]:.4f} | {score[1]:.4f}")


Epoch 1/100


c:\Users\stjia\anaconda3\envs\wheat_ml\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4751 - loss: 0.8034 - val_accuracy: 0.4762 - val_loss: 0.7869
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5057 - loss: 0.7772 - val_accuracy: 0.4881 - val_loss: 0.7659
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6700 - loss: 0.7539 - val_accuracy: 0.4881 - val_loss: 0.7504
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6668 - loss: 0.7357 - val_accuracy: 0.4821 - val_loss: 0.7371
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6871 - loss: 0.7164 - val_accuracy: 0.4762 - val_loss: 0.7280
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5334 - loss: 0.7155 - val_accuracy: 0.5000 - val_loss: 0.7184
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6220 - loss: 0.7000 - val_accuracy: 0.4762 - val_loss: 0.7172
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6474 - loss: 0.6891 - val_accuracy: 0.4821 - val_loss: 0.